# Winning Jeopardy

Jeopardy is a popular TV show in the US where participants answer questions to win money. It's been running for a few decades, and is a major force in popular culture.<br><br>The dataset is named jeopardy.csv, and contains 20000 rows from the beginning of a full dataset of Jeopardy questions, which can be downloaded from [here](https://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file).<br><br>Data Dictionary:

> <font color=blue>***Show Number***</font> -- the Jeopardy episode number of the show this question was in.<br>
> <font color=blue>***Air Date***</font> -- the date the episode aired.<br>
> <font color=blue>***Round***</font> -- the round of Jeopardy that the question was asked in. Jeopardy has several rounds as each episode progresses.<br>
> <font color=blue>***Category***</font> -- the category of the question.<br>
> <font color=blue>***Value***</font> -- the number of dollars answering the question correctly is worth.<br>
> <font color=blue>***Question***</font> -- the text of the question.<br>
> <font color=blue>***Answer***</font> -- the text of the answer.<br>

### <font color=Orange>***Aim***</font>

**Let's say we want to compete on Jeopardy, and we're looking for any edge we can get to win. In this project, we'll work with a dataset of Jeopardy questions to figure out some patterns in the questions that could help us win.**

### Introduction

- We will extract data into pandas dataframe. 
- Clean the dataset by dropping colimns with null values
- Clean the Column names

In [1]:
import pandas as pd

jeopardy = pd.read_csv('jeopardy.csv')
jeopardy.dropna(inplace=True)
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [2]:
jeopardy.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [3]:
# Removing spaces from column names
jeopardy.columns = [x.strip() for x in jeopardy.columns]
jeopardy.columns

Index(['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')

In [4]:
jeopardy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216928 entries, 0 to 216929
Data columns (total 7 columns):
Show Number    216928 non-null int64
Air Date       216928 non-null object
Round          216928 non-null object
Category       216928 non-null object
Value          216928 non-null object
Question       216928 non-null object
Answer         216928 non-null object
dtypes: int64(1), object(6)
memory usage: 13.2+ MB


### Normalizing Text

Before we begin to do analysis, we need to normalize all of the text columns (the `Question` and `Answer` columns). The idea is to lowercase words and remove punctuation so `Don't` and `don't` aren't considered to be different words while comparing them.

In [5]:
import re

def normalizing_string(string):
    string = string.lower()
    string = re.sub("[^A-Z0-9a-z\s]", "", string)
    return string

In [6]:
jeopardy['clean_question'] = jeopardy['Question'].apply(normalizing_string)
jeopardy['clean_answer'] = jeopardy['Answer'].apply(normalizing_string)
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no 2 1912 olympian football star at carlisle i...,jim thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,the city of yuma in this state has a record av...,arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,in 1963 live on the art linkletter show this c...,mcdonalds
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,signer of the dec of indep framer of the const...,john adams


### Normalizing Columns

There are some other columns to be normalized.

The `Value` column should be numeric to manipulate it more easily. We'll need to remove the dollar sign from the beginning of each value and convert the column from text to numeric.

The `Air Date` column should also be a datetime, not a string, enabling us to work with it more easily.

In [7]:
def normalizing_values(value):
    value = re.sub("[^A-Z0-9a-z\s]", "", value)
    try:
        value = int(value)
    except Exception:
        value = 0
    return value

In [8]:
jeopardy['clean_value'] = jeopardy['Value'].apply(normalizing_values)
jeopardy['Air Date'] = pd.to_datetime(jeopardy['Air Date'])
print(jeopardy.dtypes)
jeopardy.head()

Show Number                int64
Air Date          datetime64[ns]
Round                     object
Category                  object
Value                     object
Question                  object
Answer                    object
clean_question            object
clean_answer              object
clean_value                int64
dtype: object


,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus,200
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no 2 1912 olympian football star at carlisle i...,jim thorpe,200
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,the city of yuma in this state has a record av...,arizona,200
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,in 1963 live on the art linkletter show this c...,mcdonalds,200
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,signer of the dec of indep framer of the const...,john adams,200


In order to figure out whether to study past questions, study general knowledge, or not study it all, it would be helpful to figure out two things:

**How often the answer is deducible from the question.<br>
How often new questions are repeats of older questions.**

We can answer the second question by seeing how often complex words (> 6 characters) reoccur. We can answer the first question by seeing how many times words in the answer also occur in the question. 

We'll work on the first question now, and come back to the second.

---

### How often the answer is deducible from the question.

In [9]:
def ans_in_ques(row):
    split_answer = row['clean_answer'].split()
    split_question = row['clean_question'].split()

    match_count = 0

    try:
        split_answer.remove('the')
    except ValueError:
        pass

    if len(split_answer) == 0:
        return 0

    for element in split_answer:
        if element in split_question:
            match_count += 1
    return match_count/len(split_answer)

jeopardy['answer_in_question'] = jeopardy.apply(ans_in_ques, axis=1)
jeopardy['answer_in_question'].mean()

0.057921237245162335

*The answer only appears in the question about 6% of the time. This isn't a huge number, and means that we probably can't just hope that hearing a question will enable us to figure out the answer. We'll probably have to study.*

---

### How often new questions are repeats of older questions

In [10]:
question_overlap = []
overlapping_terms = []
terms_overlapped = set()
terms_used = set()

for i, rows in jeopardy.iterrows():
    split_question = rows['clean_question'].split(" ")
    split_question = [x for x in split_question if len(x) > 5]
    match_count = 0
    terms = []
    
    for item in split_question:
        if item in terms_used:
            match_count += 1
            terms.append(item)
            terms_overlapped.add(item)
        terms_used.add(item)
    
    if len(split_question) > 0:
        match_count /= len(split_question)
    
    question_overlap.append(match_count)
    overlapping_terms.append(terms)
    
jeopardy['question_overlap'] = question_overlap
jeopardy['overlapping_terms'] = overlapping_terms

jeopardy['question_overlap'].mean()

0.8740091471018069

*There is about 87% overlap between terms in new questions and terms in old questions. Since, we are only looking at a small set of questions, just single terms not phrases. It may look like relatively insignificant, but it's worth looking more into the recycling of questions.*

---

### Low-Value vs High-Value Questions

Let's say we only want to study questions that pertain to high value questions instead of low value questions. This will help us earn more money when we're on Jeopardy.

We can actually figure out which terms correspond to high-value questions using a chi-squared test. We'll first need to narrow down the questions into two categories:

Low value -- Any row where Value is less than 800.<br>
High value -- Any row where Value is greater than 800.

In [12]:
def high_or_low_value(row):
    value = 0
    if row['clean_value'] > 800:
        value = 1
    return value
    
jeopardy['high_value'] = jeopardy.apply(high_or_low_value, axis=1)
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value,answer_in_question,question_overlap,overlapping_terms,high_value
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus,200,0.0,0.0,[],0
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no 2 1912 olympian football star at carlisle i...,jim thorpe,200,0.0,0.0,[],0
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,the city of yuma in this state has a record av...,arizona,200,0.0,0.0,[],0
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,in 1963 live on the art linkletter show this c...,mcdonalds,200,0.0,0.0,[],0
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,signer of the dec of indep framer of the const...,john adams,200,0.0,0.0,[],0


*High Value column categorizes data into either High Value [1] or Low Value [0].*

---

### Observed Quantity of High Value vs Low Value Questions

In [13]:
def high_or_low_count(word):
    low_count = 0
    high_count = 0
    
    for i, rows in jeopardy.iterrows():
        split_question = rows['clean_question'].split(" ")
        if word in split_question:
            if rows['high_value'] == 1:
                high_count += 1
            else:
                low_count += 1
                
    return high_count, low_count

observed_high_low = []
comparison_terms = list(terms_overlapped)[:5]

for item in comparison_terms:
    observed_high_low.append(high_or_low_count(item))
    
observed_high_low

[(6, 4), (30, 117), (2, 0), (9, 36), (2, 4)]

In [14]:
comparison_terms

['sturdy', 'records', 'counterterrorism', 'offspring', 'corresponds']

*For terms in `comparison_terms` the Observed High Count, Low Count is mentioned in `observed_high_low`*

---

### Applying the Chi-Squared test


Now, we will find the expected High Count, Low Count.<br> 
Then, we will pass expected and observed values through `chisquare` function from `scipy.stats` to get the chi-value.

In [16]:
from scipy.stats import chisquare
import numpy as np

high_value_count = jeopardy[jeopardy['high_value'] == 1].shape[0]
low_value_count = jeopardy[jeopardy['high_value'] == 0].shape[0]

chi_squared = []

for item in observed_high_low:
    total = sum(item)
    total_prop = total/jeopardy.shape[0]
    high_value_expected = total_prop*high_value_count
    low_value_expected = total_prop*low_value_count
    
    observed = np.array([item[0], item[1]])
    expected = np.array([high_value_expected, low_value_expected])
    
    chi_squared.append(chisquare(observed, expected))
    
chi_squared

[Power_divergenceResult(statistic=4.94632037779215, pvalue=0.026146260466249117),
 Power_divergenceResult(statistic=4.527141729694901, pvalue=0.03336129348755921),
 Power_divergenceResult(statistic=5.063527726221873, pvalue=0.024434452137600038),
 Power_divergenceResult(statistic=1.5326434242696305, pvalue=0.2157161470448743),
 Power_divergenceResult(statistic=0.07446002639524918, pvalue=0.7849503458405339)]

### Chi Squared Results

First three terms of the chi-squared result show a high chi-squared value. It means that words `sturdy`, `records` and `counterterrrorism` are more common then expected. 

***If we perform the same test for all the words, then words with highest chi-square value would be most valuable to study.***